In [ ]:
# Clone FastSAM and your camera_stream script
!git clone https://github.com/demelere/FastSAM.git
!mkdir -p camera_stream
!wget https://raw.githubusercontent.com/demelere/Weld-Seam-Finding/main/camera_stream/ip_camera_stream.py -O camera_stream/ip_camera_stream.py

# Install requirements
!pip install -r FastSAM/requirements.txt

In [ ]:
import sys
sys.path.append('FastSAM')
sys.path.append('camera_stream')

import cv2
import torch
import numpy as np
from FastSAM.fastsam import FastSAM, FastSAMPrompt
from camera_stream.ip_camera_stream import connect_to_continuity_camera

In [ ]:
model = FastSAM('FastSAM.pt')
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")

In [ ]:
def process_frame(frame):
    """Process a single frame with FastSAM and overlay segmentation masks."""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    everything_results = model(
        frame_rgb, device=device, retina_masks=True, imgsz=1024, conf=0.4, iou=0.9
    )
    prompt_process = FastSAMPrompt(frame_rgb, everything_results, device=device)
    ann = prompt_process.everything_prompt()
    
    overlay = frame.copy()
    for mask in ann:
        mask = mask.astype(np.uint8)
        mask = cv2.resize(mask, (frame.shape[1], frame.shape[0]))
        color = np.random.randint(0, 255, 3).tolist()
        overlay[mask > 0] = [int(0.6 * c + 0.4 * p) for c, p in zip(color, overlay[mask > 0][0])]
    output = cv2.addWeighted(frame, 0.4, overlay, 0.6, 0)
    return output

In [ ]:
def run_realtime_segmentation():
    cap = connect_to_continuity_camera()
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Failed to grab frame")
                break
            output_frame = process_frame(frame)
            cv2.imshow('FastSAM Real-time Segmentation', output_frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
    finally:
        cap.release()
        cv2.destroyAllWindows()

In [ ]:
run_realtime_segmentation()